In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<br>
<br>
<br>
<br>

# **Feature Engineering: bureau_balance**

A Feature Engineering é um processo fundamental para ciência de dados. Ela se refere ao processo de criação e transformação de variáveis para melhorar o desempenho dos modelos de Machine Learning.
<br>
<br>
Nesse notebook vamos trabalhar na criação de variáveis para o dataset **`bureau_balance`**. Esse conjunto de dados contém informações mensais sobre créditos anteriores do cliente em outras instituições financeiras.
<br>
<br>
Com esses dados podemos gerar insights sobre o comportamento passado do cliente em relação aos pagamentos. Isso permite avaliar a pontualidade dos pagamentos, identificar padrões de pagamento e antecipar possíveis dificuldades financeiras, ajudando os credores a mitigar o risco de inadimplência e tomar decisões mais informadas sobre a concessão de crédito.
<br>
<br>
Por se tratar de criação de variáveis, nessa etapa geralmente trabalhamos com muito volume de dados e grande demanda de processamento, por essa razão iremos utilizar o **Spark** e **Spark.SQL** para essa etapa.

<br>

## **Pacotes e Bibliotecas**

In [2]:
# Instalando o PySpark.
!pip install pyspark -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
# Importando bibliotecas.
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, round, sum, avg, max, min, when, countDistinct, count, date_format, current_date
import os
import sys

In [4]:
# Configurando a sessão Spark.
spark = SparkSession.builder \
    .appName('app_spark_3') \
    .config('spark.driver.memory', '4g') \
    .config('spark.executor.memory', '4g') \
    .getOrCreate()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

<br>
<br>
<br>
<br>

## **Leitura e visualização do conjunto de dados**

In [5]:
# Lendo nosso conjunto de dados e criando um DataFrame no Spark.
dados = spark.read.csv('/content/drive/MyDrive/Projetos_Big_Data_Analytics/Ciencia_de_Dados/Etapa_Modelagem_Credito/pod-academy-analise-de-credito-para-fintech/database/bureau_balance.csv', header=True, inferSchema=True)
dados.createOrReplaceTempView('dados')

# Mostra os dados.
dados.show()

+------------+--------------+------+
|SK_ID_BUREAU|MONTHS_BALANCE|STATUS|
+------------+--------------+------+
|     5715448|             0|     C|
|     5715448|            -1|     C|
|     5715448|            -2|     C|
|     5715448|            -3|     C|
|     5715448|            -4|     C|
|     5715448|            -5|     C|
|     5715448|            -6|     C|
|     5715448|            -7|     C|
|     5715448|            -8|     C|
|     5715448|            -9|     0|
|     5715448|           -10|     0|
|     5715448|           -11|     X|
|     5715448|           -12|     X|
|     5715448|           -13|     X|
|     5715448|           -14|     0|
|     5715448|           -15|     0|
|     5715448|           -16|     0|
|     5715448|           -17|     0|
|     5715448|           -18|     0|
|     5715448|           -19|     0|
+------------+--------------+------+
only showing top 20 rows



In [6]:
# Checando a quantidade de linhas e colunas do DataFrame.

# Quantidade de linhas.
num_rows = dados.count()

# Quantidade de colunas.
num_columns = len(dados.columns)

# Quantidade de IDs únicos.
num_id_bureau = dados.select(countDistinct('SK_ID_BUREAU').alias('num_id_bureau')).collect()[0]['num_id_bureau']

# Imprimir o resultado.
print(f'Quantidade de linhas do DataFrame: {num_rows}')
print(f'Quantidade de colunas do DataFrame: {num_columns}')
print(f'Quantidade de "SK_ID_BUREAU" únicos do DataFrame: {num_id_bureau}')

Quantidade de linhas do DataFrame: 27299925
Quantidade de colunas do DataFrame: 3
Quantidade de "SK_ID_BUREAU" únicos do DataFrame: 817395


In [7]:
# Exibindo a estrutura do schema do DataFrame.
dados.printSchema()

root
 |-- SK_ID_BUREAU: integer (nullable = true)
 |-- MONTHS_BALANCE: integer (nullable = true)
 |-- STATUS: string (nullable = true)



<br>
<br>
<br>
<br>

## **Criação de flags**

A criação de flags binárias 1 ou 0 será útil para nos auxiliar na visão temporal dos dados e na criação das variáveis.
<br>
<br>
Serão criadas flags para os últimos 3, 6, 12, 24 e 36 meses e flags para cada registro da variável **`STATUS`**.

### **Flags de janela de tempo**

In [8]:
df_temp_01 = spark.sql('''

SELECT
  *,
  CASE
    WHEN MONTHS_BALANCE >=  -3 THEN 1 ELSE 0 END AS U3M,
  CASE
    WHEN MONTHS_BALANCE >=  -6 THEN 1 ELSE 0 END AS U6M,
  CASE
    WHEN MONTHS_BALANCE >= -12 THEN 1 ELSE 0 END AS U12M,
  CASE
    WHEN MONTHS_BALANCE >= -24 THEN 1 ELSE 0 END AS U24M,
  CASE
    WHEN MONTHS_BALANCE >= -36 THEN 1 ELSE 0 END AS U36M
FROM
  dados
ORDER BY
  `SK_ID_BUREAU`;

''')
df_temp_01.createOrReplaceTempView('df_temp_01')
df_temp_01.show()

+------------+--------------+------+---+---+----+----+----+
|SK_ID_BUREAU|MONTHS_BALANCE|STATUS|U3M|U6M|U12M|U24M|U36M|
+------------+--------------+------+---+---+----+----+----+
|     5001709|             0|     C|  1|  1|   1|   1|   1|
|     5001709|           -20|     C|  0|  0|   0|   1|   1|
|     5001709|            -1|     C|  1|  1|   1|   1|   1|
|     5001709|            -2|     C|  1|  1|   1|   1|   1|
|     5001709|            -3|     C|  1|  1|   1|   1|   1|
|     5001709|            -4|     C|  0|  1|   1|   1|   1|
|     5001709|            -5|     C|  0|  1|   1|   1|   1|
|     5001709|            -6|     C|  0|  1|   1|   1|   1|
|     5001709|            -7|     C|  0|  0|   1|   1|   1|
|     5001709|            -8|     C|  0|  0|   1|   1|   1|
|     5001709|            -9|     C|  0|  0|   1|   1|   1|
|     5001709|           -10|     C|  0|  0|   1|   1|   1|
|     5001709|           -11|     C|  0|  0|   1|   1|   1|
|     5001709|           -12|     C|  0|

<br>

### **Flags para a variável "STATUS"**

In [13]:
unique_status = df_temp_01.select('STATUS').distinct()
unique_status.show()

+------+
|STATUS|
+------+
|     3|
|     0|
|     5|
|     C|
|     X|
|     1|
|     4|
|     2|
+------+



In [14]:
df_temp_02 = spark.sql('''

SELECT
  *,
  CASE
    WHEN STATUS = '3' THEN 1 ELSE 0 END AS STATUS_3,
  CASE
    WHEN STATUS = '0' THEN 1 ELSE 0 END AS STATUS_0,
  CASE
    WHEN STATUS = '5' THEN 1 ELSE 0 END AS STATUS_5,
  CASE
    WHEN STATUS = 'C' THEN 1 ELSE 0 END AS STATUS_C,
  CASE
    WHEN STATUS = 'X' THEN 1 ELSE 0 END AS STATUS_X,
  CASE
    WHEN STATUS = '1' THEN 1 ELSE 0 END AS STATUS_1,
  CASE
    WHEN STATUS = '4' THEN 1 ELSE 0 END AS STATUS_4,
  CASE
    WHEN STATUS = '2' THEN 1 ELSE 0 END AS STATUS_2
FROM
  df_temp_01
ORDER BY
  `SK_ID_BUREAU`;

''')
df_temp_02.createOrReplaceTempView('df_temp_02')
df_temp_02.show()

+------------+--------------+------+---+---+----+----+----+--------+--------+--------+--------+--------+--------+--------+--------+
|SK_ID_BUREAU|MONTHS_BALANCE|STATUS|U3M|U6M|U12M|U24M|U36M|STATUS_3|STATUS_0|STATUS_5|STATUS_C|STATUS_X|STATUS_1|STATUS_4|STATUS_2|
+------------+--------------+------+---+---+----+----+----+--------+--------+--------+--------+--------+--------+--------+--------+
|     5001709|             0|     C|  1|  1|   1|   1|   1|       0|       0|       0|       1|       0|       0|       0|       0|
|     5001709|           -20|     C|  0|  0|   0|   1|   1|       0|       0|       0|       1|       0|       0|       0|       0|
|     5001709|            -1|     C|  1|  1|   1|   1|   1|       0|       0|       0|       1|       0|       0|       0|       0|
|     5001709|            -2|     C|  1|  1|   1|   1|   1|       0|       0|       0|       1|       0|       0|       0|       0|
|     5001709|            -3|     C|  1|  1|   1|   1|   1|       0|       0

<br>
<br>
<br>
<br>

## **Criação de variáveis explicativas**

Vamos criar as variáveis explicativas com base nas variáveis já existentes, usando a sumarização do status de acordo com cada janela temporal criada na etapa anterior.

In [15]:
# Definindo as colunas para a agregação.
colunas_agregacao_total = ['STATUS_3', 'STATUS_0', 'STATUS_5', 'STATUS_C', 'STATUS_X', 'STATUS_1', 'STATUS_4', 'STATUS_2']

# Defindo a lista de colunas de flags.
colunas_flags = ['U3M', 'U6M', 'U12M', 'U24M', 'U36M']

# Criando uma lista vazia.
expressoes_agregacao = []

# Iterando sobre as colunas e criando as variáveis explicativas com as agregações.
for flag in colunas_flags:
  for coluna in colunas_agregacao_total:
    expressoes_agregacao.append(round(count(when(col(flag) == 1, col(coluna))), 2).alias(f'QTD_{coluna.upper()}_{flag.upper()}'))

# Criando uma tupla com as variáveis criadas.
expressoes_agregacao = tuple(expressoes_agregacao)

# Aplicando as expressões de agregação.
df_temp_03 = df_temp_02.groupBy('SK_ID_BUREAU').agg(*expressoes_agregacao).orderBy('SK_ID_BUREAU')

# Adicionando as colunas de data ao DataFrame.
df_temp_04 = df_temp_03.withColumn('PK_DATREF', date_format(current_date(), 'yyyyMMdd')) \
                       .withColumn('PK_DAT_PROC', current_date())


# Quantidade e nome das variáveis criadas.
nomes_cols = df_temp_04.columns
nomes_cols_novas = nomes_cols[1:]
print('Quantidade Total de Variáveis Criadas:', len(df_temp_04.columns) - 3)
print('Nomes das Variáveis Criadas:', nomes_cols_novas)
print('')
print('')

# Quantidade de linhas do DataFrame.
num_rows_df = df_temp_04.count()

# Quantidade de colunas do DataFrame.
num_columns_df = len(df_temp_04.columns)

# Imprimir o resultado de número de linhas e colunas.
print(f'Quantidade de linhas do DataFrame: {num_rows_df}')
print(f'Quantidade de colunas do DataFrame: {num_columns_df}')
print('')
print('')

# Mostrando o novo DataFrame com as variáveis criadas.
df_temp_04.show(20, False)

Quantidade Total de Variáveis Criadas: 40
Nomes das Variáveis Criadas: ['QTD_STATUS_3_U3M', 'QTD_STATUS_0_U3M', 'QTD_STATUS_5_U3M', 'QTD_STATUS_C_U3M', 'QTD_STATUS_X_U3M', 'QTD_STATUS_1_U3M', 'QTD_STATUS_4_U3M', 'QTD_STATUS_2_U3M', 'QTD_STATUS_3_U6M', 'QTD_STATUS_0_U6M', 'QTD_STATUS_5_U6M', 'QTD_STATUS_C_U6M', 'QTD_STATUS_X_U6M', 'QTD_STATUS_1_U6M', 'QTD_STATUS_4_U6M', 'QTD_STATUS_2_U6M', 'QTD_STATUS_3_U12M', 'QTD_STATUS_0_U12M', 'QTD_STATUS_5_U12M', 'QTD_STATUS_C_U12M', 'QTD_STATUS_X_U12M', 'QTD_STATUS_1_U12M', 'QTD_STATUS_4_U12M', 'QTD_STATUS_2_U12M', 'QTD_STATUS_3_U24M', 'QTD_STATUS_0_U24M', 'QTD_STATUS_5_U24M', 'QTD_STATUS_C_U24M', 'QTD_STATUS_X_U24M', 'QTD_STATUS_1_U24M', 'QTD_STATUS_4_U24M', 'QTD_STATUS_2_U24M', 'QTD_STATUS_3_U36M', 'QTD_STATUS_0_U36M', 'QTD_STATUS_5_U36M', 'QTD_STATUS_C_U36M', 'QTD_STATUS_X_U36M', 'QTD_STATUS_1_U36M', 'QTD_STATUS_4_U36M', 'QTD_STATUS_2_U36M', 'PK_DATREF', 'PK_DAT_PROC']


Quantidade de linhas do DataFrame: 817395
Quantidade de colunas do DataFra

<br>
<br>
<br>
<br>

## **Salvando a tabela sumarizada**

In [16]:
df_temp_04.write.mode('overwrite').parquet('/content/drive/MyDrive/Projetos_Big_Data_Analytics/Ciencia_de_Dados/Etapa_Modelagem_Credito/pod-academy-analise-de-credito-para-fintech/feature_engineering/book_vars/book_bureau_balance')